In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from sklearn.neighbors.kde import KernelDensity
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from recomm.classifier import ClassifierNN, ClassifierDNN

ModuleNotFoundError: No module named 'scipy'

In [ ]:
init_notebook_mode(connected=True)

In [ ]:
CSV_COLUMNS = [
    "age", "workclass", "fnlwgt", "education", "education_num",
    "marital_status", "occupation", "relationship", "race", "gender",
    "capital_gain", "capital_loss", "hours_per_week", "native_country",
    "income_bracket"
]

In [ ]:
train_data = pd.read_csv("data/adult_train.csv", 
                         names=CSV_COLUMNS, 
                         skiprows=1)
test_data = pd.read_csv("data/adult_test.csv", 
                        names=CSV_COLUMNS, 
                        skiprows=1)
total_data = train_data.append(test_data)
# to unify the column income_bracket
total_data["income_bracket"] = total_data["income_bracket"].str.replace(".", "")